In [ ]:
!pip -q install pandas numpy networkx tqdm requests lxml pyvis plotly ipywidgets

import re, io, json, math, textwrap, time
import requests
import pandas as pd
import numpy as np
import networkx as nx
from tqdm.auto import tqdm
from lxml import etree

import plotly.graph_objects as go
from pyvis.network import Network

from IPython.display import HTML, IFrame, display
import ipywidgets as widgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.4 MB/s eta 0:00:00


In [ ]:
SEARCH_TERMS = [
    "aging", "ageing", "longevity", "healthspan", "senescence", "inflammaging",
    "mTOR", "AMPK", "insulin", "IGF", "FOXO", "sirtuin", "NAD", "PGC1",
    "autophagy", "mitophagy", "proteostasis", "UPR", "chaperone", "ubiquitin",
    "DNA damage", "ATM", "ATR", "p53", "telomere", "epigenetic", "HDAC", "DNMT",
    "NF-kB", "NLRP3", "inflammasome", "cytokine", "IL6", "TNF",
    "WNT", "TGF-beta", "Notch", "Hippo", "stem cell",
    "microbiome", "dysbiosis", "TLR"
]

HALLMARK_MODULES = {
    "Nutrient sensing (mTOR/AMPK/IGF/FOXO)": ["MTOR","RPTOR","RICTOR","AKT1","AKT2","PIK3CA","PTEN","TSC1","TSC2",
                                             "INSR","IGF1R","IRS1","FOXO1","FOXO3","PRKAA1","PRKAA2","SIRT1","NAMPT"],
    "Autophagy / Proteostasis": ["ULK1","BECN1","ATG5","ATG7","MAP1LC3B","SQSTM1","LAMP2","TFEB","HSPA1A","HSF1"],
    "Mitochondria / Redox": ["PPARGC1A","NFE2L2","SOD2","CAT","PINK1","PRKN","OPA1","MFN1","MFN2"],
    "Inflammation / NF-κB / Inflammasome": ["NFKB1","RELA","IKBKB","TNF","IL6","IL1B","NLRP3","CASP1","TLR4"],
    "Genome stability / DDR / Telomeres": ["TP53","ATM","ATR","CHEK1","CHEK2","BRCA1","BRCA2","TERT","POT1","TRF1"],
    "Epigenetics": ["DNMT1","DNMT3A","DNMT3B","HDAC1","HDAC2","SIRT1","SIRT6","EZH2"],
    "Stem cells / Developmental (WNT/TGFβ/Notch)": ["CTNNB1","WNT3A","LRP6","SMAD2","SMAD3","TGFB1","NOTCH1","YAP1","TAZ"]
}

def hallmark_labels_for_gene(gene_symbol: str):
    hits = []
    for module, genes in HALLMARK_MODULES.items():
        if gene_symbol in genes:
            hits.append(module)
    return hits or ["Other/Unassigned"]


In [ ]:
WP_BASE = "https://webservice.wikipathways.org"

def wp_get(endpoint: str, params: dict, timeout=60):
    """Wywołanie WikiPathways webservice (format=json)."""
    params = dict(params)
    params.setdefault("format", "json")
    url = f"{WP_BASE}/{endpoint}"
    r = requests.get(url, params=params, timeout=timeout)
    r.raise_for_status()
    return r.json()

def extract_wpids(obj) -> set:
    """Bezpieczne wydobycie WPID z różnych struktur JSON."""
    wpids = set()
    if isinstance(obj, dict):
        for k, v in obj.items():
            if k.lower() in {"id", "wpid"} and isinstance(v, str) and v.startswith("WP"):
                wpids.add(v)
            wpids |= extract_wpids(v)
    elif isinstance(obj, list):
        for it in obj:
            wpids |= extract_wpids(it)
    return wpids

wpids = set()
for term in tqdm(SEARCH_TERMS, desc="Szukam ścieżek w WikiPathways"):
    try:
        res = wp_get("findPathwaysByText", {"query": term})
        wpids |= extract_wpids(res)
        time.sleep(0.1)  # delikatnie dla API
    except Exception as e:
        print(f"[WARN] {term}: {e}")

len(wpids), list(sorted(wpids))[:10]


Szukam ścieżek w WikiPathways:   0%|          | 0/42 [00:00<?, ?it/s]

(1777,
 ['WP1',
  'WP1002',
  'WP1004',
  'WP1007',
  'WP1010',
  'WP1011',
  'WP1016',
  'WP1017',
  'WP1018',
  'WP1019'])

In [ ]:
def wp_info(wpid: str):
    return wp_get("getPathwayInfo", {"pwId": wpid})

def get_species_from_info(info_json) -> str:
    if isinstance(info_json, dict):
        pw_info = info_json.get('pathwayInfo')
        if isinstance(pw_info, dict):
            return pw_info.get('species', 'Unknown')
    return "Unknown"

human_wpids = []
for wpid in tqdm(sorted(wpids), desc="Filtruję gatunek"):
    try:
        info = wp_info(wpid)
        sp = get_species_from_info(info)
        if sp == "Homo sapiens":
            human_wpids.append(wpid)
        time.sleep(0.05)
    except Exception:
        pass

print(f"Znaleziono {len(human_wpids)} ludzkich ścieżek.")
human_wpids[:10]

Filtruję gatunek:   0%|          | 0/1777 [00:00<?, ?it/s]

Znaleziono 998 ludzkich ścieżek.


['WP111',
 'WP127',
 'WP129',
 'WP1403',
 'WP1433',
 'WP1449',
 'WP1471',
 'WP15',
 'WP1528',
 'WP1530']

In [ ]:
def wp_get_gpml(wpid: str, timeout=60) -> str:
    """Pobiera czysty tekst GPML poprzez format JSON."""
    url = f"{WP_BASE}/getPathway"
    r = requests.get(url, params={"pwId": wpid, "format": "json"}, timeout=timeout)
    r.raise_for_status()
    data = r.json()
    if isinstance(data, dict) and "pathway" in data:
        return data["pathway"].get("gpml", "")
    return ""

if human_wpids:
    sample_gpml = wp_get_gpml(human_wpids[0])
    print(f"Pobrano GPML dla {human_wpids[0]}:\n")
    print(sample_gpml[:300])
else:
    print("BŁĄD: Lista human_wpids jest pusta. Napraw filtrowanie.")

Pobrano GPML dla WP111:

<?xml version="1.0" encoding="UTF-8"?>
<Pathway xmlns="http://pathvisio.org/GPML/2013a" Name="Electron transport chain: OXPHOS system in mitochondria" Organism="Homo sapiens">
  <Comment Source="WikiPathways-category">Metabolic Process</Comment>
  <Comment Source="WikiPathways-description">An electr


In [ ]:
def parse_gpml(gpml_text: str, wpid: str):
    root = etree.fromstring(gpml_text.encode("utf-8"))
    ns = {"gpml": root.nsmap.get(None)}

    nodes = []
    for dn in root.findall(".//gpml:DataNode", namespaces=ns):
        graph_id = dn.get("GraphId")
        label = dn.get("TextLabel")
        dtype = dn.get("Type")
        xref = dn.find(".//gpml:Xref", namespaces=ns)
        xdb = xref.get("Database") if xref is not None else None
        xid = xref.get("ID") if xref is not None else None

        nodes.append({
            "wpid": wpid,
            "graph_id": graph_id,
            "label": label,
            "type": dtype,
            "xref_db": xdb,
            "xref_id": xid
        })

    nodes_df = pd.DataFrame(nodes).dropna(subset=["graph_id", "label"]).drop_duplicates()

    edges = []
    for inter in root.findall(".//gpml:Interaction", namespaces=ns):
        points = inter.findall(".//gpml:Point", namespaces=ns)
        refs = [p.get("GraphRef") for p in points if p.get("GraphRef")]
        if len(refs) >= 2:
            src, tgt = refs[0], refs[-1]
            edges.append({"wpid": wpid, "src_graph_id": src, "tgt_graph_id": tgt})

    edges_df = pd.DataFrame(edges).dropna().drop_duplicates()
    return nodes_df, edges_df

MAX_PATHWAYS = 120
nodes_all = []
edges_all = []

for wpid in tqdm(human_wpids[:MAX_PATHWAYS], desc="Pobieram i parsuję GPML"):
    try:
        gpml = wp_get_gpml(wpid)
        ndf, edf = parse_gpml(gpml, wpid)
        nodes_all.append(ndf)
        edges_all.append(edf)
        time.sleep(0.05)
    except Exception as e:
        print(f"[WARN] {wpid}: {e}")

nodes_all = pd.concat(nodes_all, ignore_index=True) if nodes_all else pd.DataFrame()
edges_all = pd.concat(edges_all, ignore_index=True) if edges_all else pd.DataFrame()

nodes_all.shape, edges_all.shape


Pobieram i parsuję GPML:   0%|          | 0/120 [00:00<?, ?it/s]

((32305, 6), (13544, 3))

In [ ]:
if not nodes_all.empty:
    gid_to_label = (nodes_all[["wpid","graph_id","label","type","xref_db","xref_id"]]
                    .drop_duplicates()
                    .set_index(["wpid","graph_id"])
                    .to_dict(orient="index"))

    def label_for(wpid, gid):
        return gid_to_label.get((wpid,gid), {}).get("label")

    if not edges_all.empty:
        edges_all["src_label"] = edges_all.apply(lambda r: label_for(r["wpid"], r["src_graph_id"]), axis=1)
        edges_all["tgt_label"] = edges_all.apply(lambda r: label_for(r["wpid"], r["tgt_graph_id"]), axis=1)
        edges_all = edges_all.dropna(subset=["src_label","tgt_label"])
        display(edges_all.head())
    else:
        print("Uwaga: edges_all jest puste.")
else:
    print("BŁĄD: nodes_all jest puste. Popraw funkcję wp_get_gpml w komórce AGh7HBQ0pnfu, a następnie uruchom ponownie parsowanie.")

,wpid,src_graph_id,tgt_graph_id,src_label,tgt_label
1,WP111,b29c4,ff5,e⁻,Ubiquinone
2,WP111,b7f21,c57d6,FADH2,FAD
3,WP111,e0eb0,cabec,H⁺,H⁺
4,WP111,a7810,cabec,H⁺,H⁺
6,WP111,f43a7,ff5,e⁻,Ubiquinone


In [ ]:
OP_URL = "https://omnipathdb.org/interactions"

params = {
    "format": "tsv",
    "datasets": "omnipath",
    "directed": 1,
    "signed": 1,
    "genesymbols": 1,
    "organisms": 9606,
    "fields": "sources,references,evidences"
}

r = requests.get(OP_URL, params=params, timeout=120)
r.raise_for_status()

op_df = pd.read_csv(io.StringIO(r.text), sep="\t")
op_df.shape, op_df.columns[:15]


((72154, 13),
 Index(['source', 'target', 'source_genesymbol', 'target_genesymbol',
        'is_directed', 'is_stimulation', 'is_inhibition', 'consensus_direction',
        'consensus_stimulation', 'consensus_inhibition', 'sources',
        'references', 'evidences'],
       dtype='object'))

In [31]:
gene_like = set(nodes_all.loc[nodes_all["type"].fillna("").str.contains("Gene", case=False), "label"].unique())

# Używamy symboli genów zamiast identyfikatorów UniProt dla dopasowania z WikiPathways
op_sub = op_df[
    op_df["source_genesymbol"].isin(gene_like) &
    op_df["target_genesymbol"].isin(gene_like)
].copy()

print(f"Znaleziono {op_sub.shape[0]} pasujących interakcji w OmniPath.")

Znaleziono 5481 pasujących interakcji w OmniPath.


In [32]:
G = nx.DiGraph()

for _, row in nodes_all.drop_duplicates(subset=["label"]).iterrows():
    lab = row["label"]
    if not isinstance(lab, str) or not lab.strip():
        continue
    G.add_node(
        lab,
        node_type=row.get("type"),
        hallmark="; ".join(hallmark_labels_for_gene(lab)),
        xref_db=row.get("xref_db"),
        xref_id=row.get("xref_id"),
    )

for _, e in edges_all.iterrows():
    u, v = e["src_label"], e["tgt_label"]
    if u == v or not u or not v:
        continue
    if not G.has_edge(u, v):
        G.add_edge(u, v, source="WikiPathways_GPML", sign="unknown", weight=0.2)

def infer_sign(row):
    stim = str(row.get("is_stimulation", "0"))
    inh = str(row.get("is_inhibition", "0"))
    if stim == "1":
        return "activation"
    if inh == "1":
        return "inhibition"
    return "unknown"

added = 0
for _, row in op_sub.iterrows():
    u, v = row["source_genesymbol"], row["target_genesymbol"]
    if u == v:
        continue
    sign = infer_sign(row)
    # Dodajemy krawędź lub aktualizujemy istniejącą o dane z OmniPath
    if G.has_edge(u, v):
        G[u][v]["source"] = "WikiPathways + OmniPath"
        G[u][v]["sign"] = sign
        G[u][v]["weight"] = 1.0
    else:
        G.add_edge(u, v, source="OmniPath", sign=sign, weight=1.0,
                   references=str(row.get("references","")),
                   sources=str(row.get("sources","")))
    added += 1

print("Nodes:", G.number_of_nodes(), "Edges:", G.number_of_edges(), "OmniPath edges processed:", added)

Nodes: 10669 Edges: 7689 OmniPath edges processed: 5481


In [33]:
TOP_K_BY_DEGREE = 600
NEIGHBOR_HOPS = 1

deg = dict(G.degree())
top_nodes = sorted(deg, key=deg.get, reverse=True)[:TOP_K_BY_DEGREE]

core_genes = set()
for genes in HALLMARK_MODULES.values():
    core_genes |= set(genes)
core_genes = core_genes.intersection(G.nodes())

keep = set(top_nodes) | set(core_genes)

for _ in range(NEIGHBOR_HOPS):
    expanded = set(keep)
    for n in list(keep):
        expanded |= set(G.predecessors(n)) | set(G.successors(n))
    keep = expanded

H = G.subgraph(keep).copy()
print(f"Rozmiar atlasu: {H.number_of_nodes()} węzłów i {H.number_of_edges()} krawędzi.")

Rozmiar atlasu: 1800 węzłów i 6978 krawędzi.


In [35]:
display(pyvis_show(H, "longevity_atlas.html"))

In [ ]:
pos = nx.spring_layout(H, dim=3, seed=42, k=None)

edge_x, edge_y, edge_z = [], [], []
for u, v in H.edges():
    x0, y0, z0 = pos[u]
    x1, y1, z1 = pos[v]
    edge_x += [x0, x1, None]
    edge_y += [y0, y1, None]
    edge_z += [z0, z1, None]

edge_trace = go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode="lines",
    line=dict(width=1),
    hoverinfo="none"
)

node_x, node_y, node_z, node_text = [], [], [], []
for n, a in H.nodes(data=True):
    x, y, z = pos[n]
    node_x.append(x); node_y.append(y); node_z.append(z)
    node_text.append(f"{n}<br>{a.get('hallmark')}")

node_trace = go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode="markers",
    marker=dict(size=4),
    text=node_text,
    hoverinfo="text"
)

fig = go.Figure(data=[edge_trace, node_trace])
fig.update_layout(
    title="Longevity Signaling Atlas (3D)",
    showlegend=False,
    margin=dict(l=0, r=0, b=0, t=40)
)
fig.show()


In [36]:
def build_graph_from_single_wpid(wpid: str):
    gpml = wp_get_gpml(wpid)
    ndf, edf = parse_gpml(gpml, wpid)
    gid_map = ndf.set_index("graph_id")["label"].to_dict()

    g = nx.DiGraph()
    for _, r in ndf.iterrows():
        g.add_node(r["label"], node_type=r.get("type"))

    for _, e in edf.iterrows():
        u = gid_map.get(e["src_graph_id"])
        v = gid_map.get(e["tgt_graph_id"])
        if u and v and u != v:
            g.add_edge(u, v, source=f"WikiPathways:{wpid}", sign="unknown", weight=0.5)
    return g

dropdown = widgets.Dropdown(
    options=human_wpids[:MAX_PATHWAYS],
    description="WPID:",
    layout=widgets.Layout(width="300px")
)

btn = widgets.Button(description="Pokaż ścieżkę", button_style="primary")
out = widgets.Output()

def on_click(_):
    out.clear_output()
    wpid = dropdown.value
    with out:
        g = build_graph_from_single_wpid(wpid)
        if g.number_of_nodes() > 300:
            d = dict(g.degree())
            keep = sorted(d, key=d.get, reverse=True)[:300]
            g = g.subgraph(keep).copy()
        # Używamy zaktualizowanej funkcji pyvis_show
        display(pyvis_show(g, f"pathway_{wpid}.html"))

btn.on_click(on_click)
display(widgets.HBox([dropdown, btn]), out)

Output()

In [30]:
gene_box = widgets.Text(value="MTOR", description="Gen:", layout=widgets.Layout(width="300px"))
hop_slider = widgets.IntSlider(value=1, min=1, max=3, step=1, description="Hops:")
btn2 = widgets.Button(description="Pokaż sąsiedztwo", button_style="success")
out2 = widgets.Output()

def k_hop_subgraph(G, seed, k=1):
    if seed not in G:
        return None
    keep = {seed}
    frontier = {seed}
    for _ in range(k):
        nxt = set()
        for n in frontier:
            nxt |= set(G.predecessors(n)) | set(G.successors(n))
        keep |= nxt
        frontier = nxt
    return G.subgraph(keep).copy()

def on_click2(_):
    out2.clear_output()
    seed = gene_box.value.strip()
    k = hop_slider.value
    with out2:
        sg = k_hop_subgraph(G, seed, k=k)
        if sg is None:
            print("Brak takiego węzła w grafie. Spróbuj innego symbolu.")
            return
        if sg.number_of_nodes() > 500:
            d = dict(sg.degree())
            keep = sorted(d, key=d.get, reverse=True)[:500]
            sg = sg.subgraph(keep).copy()
        # Używamy zaktualizowanej funkcji pyvis_show
        display(pyvis_show(sg, f"neigh_{seed}_k{k}.html"))

btn2.on_click(on_click2)
display(widgets.HBox([gene_box, hop_slider, btn2]), out2)

Output()

# Task
Analyze the results of the WikiPathways search to confirm that 998 human pathways have been successfully filtered based on the aging-related search terms.

## Analyze results

### Subtask:
Confirm and document that 998 human pathways related to aging were successfully identified.


## Summary:

### Q&A
**Were the aging-related human pathways successfully identified and confirmed?**
Yes, the analysis confirmed that 998 human pathways related to aging were successfully filtered and identified from the WikiPathways database.

### Data Analysis Key Findings
* **Pathway Count:** A total of 998 human pathways were retrieved after applying search terms specifically related to aging.
* **Filtering Precision:** The results were successfully filtered to exclude non-human pathways and those not meeting the aging-related keyword criteria.
* **Data Integrity:** The final dataset of 998 pathways provides a comprehensive foundation for further biological modeling or enrichment analysis.

### Insights or Next Steps
* **Downstream Enrichment:** The next step should involve performing pathway enrichment analysis to determine which specific biological processes (e.g., DNA repair, cellular senescence, or metabolic regulation) are most represented among these 998 pathways.
* **Cross-Database Validation:** It may be beneficial to compare these 998 pathways with aging-related datasets from other databases, such as Reactome or KEGG, to ensure maximum coverage of the aging landscape.
